In [2]:
import geopandas as gpd
import pandas as pd
import pandas as pd
import geopandas
import shapely.geometry as geom
from polygon_geohasher.polygon_geohasher import polygon_to_geohashes, geohashes_to_polygon

%matplotlib inline
import geohash
import geopandas as gp
import pandas as pd
import math
import geojson
from geojson import MultiLineString
from shapely.geometry import MultiLineString
from shapely.geometry import MultiPoint
import numpy as np
from shapely import geometry
from shapely.geometry import Point, Polygon, box,LineString
from geopandas import datasets, GeoDataFrame, read_file
from geopandas.tools import overlay
from matplotlib import pyplot
import matplotlib.pyplot as plt
from shapely.geometry.polygon import LinearRing, Polygon
from numpy import cos,sin,arccos
import numpy as np
from pylab import *
from shapely.ops import cascaded_union
from shapely.geometry import Point, LineString, mapping
import scipy.interpolate
import geopy
import geopy.distance
import time


import networkx as nx

In [13]:
def compute_geohash_tiles_from_polygon(polygon,GEOHASH_PRECISION):
    """Computes all hex tile in the given polygon

    :param polygon: the polygon
    :return: a list of geohashes
    """

    checked_geohashes = set()
    geohash_stack = set()
    geohashes = []
    # get center of bounding, assuming the earth is flat ;)
    center_latitude = polygon.centroid.coords[0][1]
    center_longitude = polygon.centroid.coords[0][0]

    center_geohash = geohash.encode(center_latitude, center_longitude, precision=GEOHASH_PRECISION)
    geohashes.append(center_geohash)
    geohash_stack.add(center_geohash)
    checked_geohashes.add(center_geohash)
    while len(geohash_stack) > 0:
        current_geohash = geohash_stack.pop()
        neighbors = geohash.neighbors(current_geohash)
        for neighbor in neighbors:
            point = geometry.Point(geohash.decode(neighbor)[::-1])
            if neighbor not in checked_geohashes and polygon.contains(point):
                geohashes.append(neighbor)
                geohash_stack.add(neighbor)
                checked_geohashes.add(neighbor)
    return geohashes


def splitDataFrameList(df,target_column,separator):
    import pandas
    ''' df = dataframe to split,
    target_column = the column containing the values to split
    separator = the symbol used to perform the split
    returns: a dataframe with each entry for the target column separated, with each element moved into a new row. 
    The values in the other columns are duplicated across the newly divided rows.
    '''
    def splitListToRows(row,row_accumulator,target_column,separator):
        split_row = row[target_column].split(separator)
        for s in split_row:
            new_row = row.to_dict()
            new_row[target_column] = s
            row_accumulator.append(new_row)
    new_rows = []
    df.apply(splitListToRows,axis=1,args = (new_rows,target_column,separator))
    new_df = pandas.DataFrame(new_rows)
    return new_df

In [9]:
merge_road = gp.GeoDataFrame.from_file('results/selagor_roads/selangor_roads.shp', geometry='geometry')
merge_road.head()

,osm_id_roa,code,fclass,name,ref,oneway,maxspeed,layer,bridge,tunnel,...,NL_NAME_1,TYPE_1,ENGTYPE_1,CC_1,HASC_1,lat,lon,geohash,osm_id_poi,geometry
0,4931450,5122,residential,Jalan Pinggiran USJ 1/17,None,B,0,0,F,F,...,None,Negeri,State,None,MY.SL,3.050336,101.558314,w281r52,1.0,"LINESTRING (101.55520 3.05020, 101.55625 3.050..."
1,4931468,5115,tertiary,Jalan Pinggiran USJ 1,None,B,0,0,F,F,...,None,Negeri,State,None,MY.SL,3.049863,101.557853,w281r50,0.0,"LINESTRING (101.55488 3.05006, 101.55509 3.050..."
2,4931834,5122,residential,Persiaran Perusahaan U/1,None,B,0,0,F,F,...,None,Negeri,State,None,MY.SL,3.088406,101.554642,w281wfj,0.0,"LINESTRING (101.55467 3.09144, 101.55465 3.090..."
3,4931877,5113,primary,Jalan Montford,B9,F,0,0,F,F,...,None,Negeri,State,None,MY.SL,3.087199,101.552459,w281wfh,0.0,"LINESTRING (101.55272 3.08739, 101.55219 3.08701)"
4,4934921,5113,primary,Bulatan Megawati,None,F,0,0,F,F,...,None,Negeri,State,None,MY.SL,3.052685,101.543664,w281qet,0.0,"LINESTRING (101.54352 3.05211, 101.54337 3.052..."


In [10]:
from shapely.geometry import shape
from shapely.ops import unary_union
import fiona
geoms =[shape(feature['geometry']) for feature in fiona.open("results/selagor_roads/selangor_roads.shp")]
weights = merge_road['osm_id_poi']
import itertools
# create a Graph
import networkx as nx
G = nx.Graph()
counter = 0
for line in geoms:
    if line.geom_type== "MultiLineString":
        for line_part in line:
            #print(line_part)
            for seg_start, seg_end in zip(list(line_part.coords),list(line_part.coords)[1:]):
                G.add_nodes_from([seg_start, seg_end])
                G.add_edge(seg_start, seg_end, weight=float(weights[counter]))
                
    else:
        #print(line)
        #print(line.coords)
    
        for seg_start, seg_end in zip(list(line.coords),list(line.coords)[1:]):
            G.add_nodes_from([seg_start, seg_end])
            G.add_edge(seg_start, seg_end, weight=float(weights[counter])) 
            
    counter += 1

In [11]:
list(G.nodes)

[(101.5552027, 3.0501994),
 (101.5562486, 3.0500211),
 (101.556847, 3.0499153),
 (101.5570107, 3.0498935),
 (101.5572537, 3.0498478),
 (101.557493, 3.0497979),
 (101.5582923, 3.0496763),
 (101.5587598, 3.049596),
 (101.5588407, 3.0496144),
 (101.5590161, 3.0498156),
 (101.5592535, 3.0500724),
 (101.5597251, 3.050633),
 (101.5602334, 3.0512056),
 (101.5603128, 3.0514655),
 (101.5603465, 3.05174),
 (101.5604314, 3.0522589),
 (101.5548825, 3.0500569),
 (101.5550899, 3.0500209),
 (101.5551143, 3.0500166),
 (101.5552623, 3.0499909),
 (101.5568117, 3.0497271),
 (101.556873, 3.0497112),
 (101.5574543, 3.0495928),
 (101.5587815, 3.0493665),
 (101.5589368, 3.0493978),
 (101.5591139, 3.0495558),
 (101.5593955, 3.0498799),
 (101.5599363, 3.0504603),
 (101.5603733, 3.0509799),
 (101.5546656, 3.091444),
 (101.5546525, 3.0906138),
 (101.5546558, 3.0898678),
 (101.554657, 3.0895888),
 (101.5546444, 3.0889426),
 (101.5546217, 3.0871187),
 (101.5546244, 3.0862943),
 (101.5546173, 3.085729),
 (101.55463

In [17]:
origin = (101.5787128, 3.0231652)
destination = (101.5261975, 3.1205146)

route1 = nx.shortest_path(G, origin, destination)
route2 = nx.dijkstra_path(G, origin, destination)
#print(route1)
#print(route2)
route1_arr_lat = []
route1_arr_lon = []
route2_arr_lat = []
route2_arr_lon = []

for element in route1:
    route1_arr_lat.append(element[1])
    route1_arr_lon.append(element[0])
    
for element in route2:
    route2_arr_lat.append(element[1])
    route2_arr_lon.append(element[0])


import geopandas as gpd
from shapely.geometry import Polygon
#print(len(route1_arr_lat))
#print(len(route1_arr_lon))
#print(route1_arr_lat)
#print(route1_arr_lon)
polygon_geom = LineString(zip(route1_arr_lon, route1_arr_lat))
crs = {'init': 'epsg:4326'}
polygon = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom])
polygon['geohash_set'] = np.nan
polygon['geohash_set'] = polygon.apply(lambda x: compute_geohash_tiles_from_polygon(x.geometry, 7), axis=1)
polygon['geohash_set'] = polygon['geohash_set'].astype(str)
polygon['geohash_set'] = polygon['geohash_set'].str[1:]
polygon['geohash_set'] = polygon['geohash_set'].str[:-1]
polygon = splitDataFrameList(polygon,'geohash_set',',')
polygon = geopandas.GeoDataFrame(polygon, geometry='geometry')
polygon.to_file(filename='sample_route_1/route3.shp')

polygon_geom = LineString(zip(route2_arr_lon, route2_arr_lat))
crs = {'init': 'epsg:4326'}
polygon = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom])
polygon['geohash_set'] = np.nan
polygon['geohash_set'] = polygon.apply(lambda x: compute_geohash_tiles_from_polygon(x.geometry, 7), axis=1)
polygon['geohash_set'] = polygon['geohash_set'].astype(str)
polygon['geohash_set'] = polygon['geohash_set'].str[1:]
polygon['geohash_set'] = polygon['geohash_set'].str[:-1]
polygon = splitDataFrameList(polygon,'geohash_set',',')
polygon = geopandas.GeoDataFrame(polygon, geometry='geometry')
polygon.to_file(filename='sample_route_2/route4.shp')